**This is my first public Kaggle notebook!** I am happy to share my experience with you, Kagglers!

**My roadmap:**

* Importing dataset,
* Visualization dataset,
* Filling missing values,
* Implementing LogisticRegression, 
* Use LogisticRegression to predict test dataset,
* Upload my results to public leaderboard.



In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
df.head()

In [ ]:
test.tail()

*df.head()* & *df.tail()* made similar work!

First of all, I must change **Sex** from **Categorical format** to **Numerical format** in both *df* & *test*:

In [ ]:
sex_bin = {"female": 0,   # Zero is female 
           "male": 1}     # One is for male
df['Sex'] = df['Sex'].map(sex_bin)
test['Sex'] = test['Sex'].map(sex_bin)

In [ ]:
sns.countplot(x="Sex", hue="Survived", data=df)

So to conclude from above plot:
* majority of male are NOT survived
* majority of female are survived
* male are greater than female

In [ ]:
sns.heatmap(data=df.corr(), annot=True )

Let\`s observe **heatmap**
* Low correlations are between Survived and [Fare & Pclass]
* Zero correlations are between Survived and [Age, SibSp, PassengerId and Parch]
* Age & Pclass has normal negative correlation which means that Pclass maybe been devived into Age category
* Fare & Pclass also have normolized negative correlation which is probably means relationship
* Parch and SibSp have normal positive correlation value
* PassengerId does not have any high or low correlation with other feature -> so we drop PassenderId

**Those correlation results tell us that Zero correlation features should be eliminated.
However, Age i think must stay for modelling.**

In [ ]:
df.info()

**Age** has many missing values!

**Cabin** has only about 25% out of 100% values!

**Embarked** has double missing values!

My plan for replacing **Age** is to use mean() function for each title in **Name**

I will remove **Cabin** from dataframe because I find it  irrelevant

I will use  the most popular value instead of  missing values in **Embarked**


In [ ]:
def get_title(name):
    first = name.split(',')
    second = first[1].split('.')
    return second[0]


In [ ]:
df.Name = df.Name.apply(get_title)
test.Name = test.Name.apply(get_title)

We REPLACED NAME TO TITLE:
* 17 UNIQUE TITLES
* TITLES ARE HELPFUL FOR PREDICTING AGE 
* TITLE HELPS TO PREDICT OTHER MISSING VALUES
* NOW I AM ANALYSING TITLES AND OTHER FEATURES


In [ ]:
def best_age(name):
    return df.loc[(df.Name == name)].Age.mean()

In [ ]:
for i in df.Name.unique():
    if (df.loc[(df.Name == i)].Age.isnull().sum()) > 0:
        print(i, "&", df.loc[(df.Name == i)].Age.isnull().sum())
        print(best_age(i))

Here is values for missing values in **Age** column

In [ ]:
df.Age=df.Age.fillna(df.Name.apply(best_age))
test.Age=test.Age.fillna(test.Name.apply(best_age))

In [ ]:
df.isnull().sum()

* Age column has many missing values 
* Cabin column also has many missing values 
* Embarked column has 2 missing values
Now i should replace missing value to reallistic values:


In [ ]:
test.isnull().sum()

In **test** dataframe, i have one missing value in **Fare** & 327 inn **Cabin**

In [ ]:
y_train = df.Survived 
X_train = df.drop("Survived", axis=1) # remove Survived from dataframe


In [ ]:
X_train = X_train.drop(["Ticket", "Cabin", "PassengerId"], axis=1) # removing other 3 figures from dataset

In [ ]:
X_train.columns

In [ ]:
X_train.loc[(X_train.Embarked.isnull())]

*As you see, We have two missing value in Embarked for Miss and Mrs*

In [ ]:
X_train.loc[(X_train.Name == " Miss") & (X_train.SibSp == 0) & (X_train.Parch== 0) & (X_train.Embarked == "S")].count()  # most popular -> "S" => X_train.Embarked == "S"  

In [ ]:
X_train.loc[(X_train.Name == " Mrs") & (X_train.SibSp == 0) & (X_train.Parch== 0) & (X_train.Embarked == "S")]

In [ ]:
X_train.Embarked.fillna("S", inplace=True)

In [ ]:
test.Fare.fillna(test.Fare.mean(), inplace=True) # fill missing values in test dataframe

In [ ]:
features = ['Pclass', 'Sex', 'Age' ,'SibSp', 'Parch', 'Fare', 'Embarked']
X = pd.get_dummies(X_train[features], drop_first=True) #change categorical variables into dummy variables 

In [ ]:
test_features = ['Pclass', 'Sex', 'Age' ,'SibSp', 'Parch', 'Fare', 'Embarked']
X_test = pd.get_dummies(test[test_features], drop_first=True)

In [ ]:
X.head()

In [ ]:
X_test.head()

In [ ]:
X_test.info()

In [ ]:
X.info()

**We finished our tasks on both dataframes**

Those dataframes are ready for fitting!

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=7, solver='liblinear') # i have used solver as a liblinear because i have binary values like 0 and 1
logreg.fit(X, y_train)


In [ ]:
logreg.score(X, y_train)

My model score is **0.79910**, when i have implemented **LogisticRegression**

In [ ]:
preds = logreg.predict(X_test)

In [ ]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': preds})
output.to_csv('LogisticRegressionv1.csv', index=False)

Thank you, Kagglers!
Share your ideas abour my notebook and upvote if it was useful!
Happy New Year to everyone!